In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from scipy.sparse import csr_matrix, vstack, hstack
from sklearn.metrics import roc_auc_score


In [2]:
train = pd.read_pickle("data/train.pkl").dropna()
test = pd.read_pickle("data/test.pkl").dropna()
y_train = np.array(train['label'])
y_test = np.array(test['label'])

In [4]:
train_parent_tdidf_csr = vstack(train["parent_comment_tdidf"])
test_parent_tdidf_csr = vstack(test["parent_comment_tdidf"])

train_tdidf_csr = vstack(train["comment_tdidf_nn"])
test_tdidf_csr = vstack(test["comment_tdidf_nn"])

train_parent_bow_csr = vstack(train["parent_comment_bow"])
test_parent_bow_csr = vstack(test["parent_comment_bow"])

train_bow_csr = vstack(train["comment_bow"])
test_bow_csr = vstack(test["comment_bow"])

In [5]:
train_tdidf_csr

<161010x8160 sparse matrix of type '<class 'numpy.float64'>'
	with 855934 stored elements in Compressed Sparse Row format>

In [6]:
list_of_features = [
       'comment_word_count','parent_comment_word_count',
       'comment_token_count', 'parent_comment_token_count',
       'comment_unique_word_count', 'parent_comment_unique_word_count',
       'comment_unique_token_count', 'parent_comment_unique_token_count',
       'comment_stopword_count', 'parent_comment_stopword_count',
       'comment_mean_word_length', 'parent_comment_mean_word_length',
       'comment_mean_token_length', 'parent_comment_mean_token_length',
       'comment_char_count', 'parent_comment_char_count',
       'comment_punctuation_count', 'parent_comment_punctuation_count',
       'comment_hashtag_count', 'parent_comment_hashtag_count',
       'comment_number_count', 'parent_comment_number_count',
       'weighted_parent_sentiment_score_neutral',
       'weighted_parent_sentiment_score_positive',
       'weighted_comment_sentiment_score_neutral',
       'weighted_comment_sentiment_score_positive']
bool_cols = ['weighted_parent_sentiment_score_neutral',
             'weighted_parent_sentiment_score_positive',
             'weighted_comment_sentiment_score_neutral',
             'weighted_comment_sentiment_score_positive']

for col in bool_cols: #need to convert bool type to integer
    train[col] = train[col].astype(int)
    test[col] = test[col].astype(int)
X_train_gen_features = csr_matrix(train[list_of_features])

X_test_gen_features = csr_matrix(test[list_of_features])
x_train = hstack([X_train_gen_features,train_tdidf_csr]).toarray() #deep learning automates feature selection. from our supervised learning we have learnt that BoW adds no information given tf-idf.
x_test = hstack([X_test_gen_features,test_tdidf_csr]).toarray()

Feedforward NN

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(128, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=8, validation_split=0.1)
y_pred_test = model.predict(x_test)
roc_auc_test = roc_auc_score(y_test, y_pred_test)
print(f'AUC: {roc_auc_test}')
#comment td-idf only AUC: 0.6628971516171749
#gen features + comment td-idf AUC: 0.7419122793471202


Epoch 1/10
18112/18112 [==============================] - 58s 3ms/step - loss: 0.6692 - accuracy: 0.5858 - val_loss: 0.6569 - val_accuracy: 0.5915
Epoch 2/10
18112/18112 [==============================] - 55s 3ms/step - loss: 0.6233 - accuracy: 0.6489 - val_loss: 0.6263 - val_accuracy: 0.6326
Epoch 3/10
18112/18112 [==============================] - 53s 3ms/step - loss: 0.6036 - accuracy: 0.6683 - val_loss: 0.6144 - val_accuracy: 0.6545
Epoch 4/10
18112/18112 [==============================] - 53s 3ms/step - loss: 0.5922 - accuracy: 0.6791 - val_loss: 0.5957 - val_accuracy: 0.6740
Epoch 5/10
18112/18112 [==============================] - 55s 3ms/step - loss: 0.5863 - accuracy: 0.6861 - val_loss: 0.5963 - val_accuracy: 0.6743
Epoch 6/10
18112/18112 [==============================] - 53s 3ms/step - loss: 0.5803 - accuracy: 0.6921 - val_loss: 0.5942 - val_accuracy: 0.6786
Epoch 7/10
18112/18112 [==============================] - 53s 3ms/step - loss: 0.5755 - accuracy: 0.6954 - val_loss: 0

using iterative hyperparameter tuning

In [7]:
from keras_tuner import HyperModel
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
#use tf.keras.optimizers.legacy.Adam if on M1/M2 macbook
class MyHyperModel(HyperModel):
    def __init__(self, input_dim):
        self.input_dim = input_dim

    def build(self, hp):
        model = Sequential()
        # First layer
        model.add(Dense(units=hp.Int('units_first', min_value=128, max_value=512, step=32),
                        activation='relu', input_dim=self.input_dim))
        model.add(Dropout(rate=hp.Float('dropout_first', min_value=0.1, max_value=0.5, step=0.1)))
        
        # Second layer
        model.add(Dense(units=hp.Int('units_second', min_value=64, max_value=256, step=32),
                        activation='relu'))
        model.add(Dropout(rate=hp.Float('dropout_second', min_value=0.1, max_value=0.5, step=0.1)))

        # Third layer
        model.add(Dense(units=hp.Int('units_third', min_value=32, max_value=128, step=32),
                        activation='relu'))
        model.add(Dropout(rate=hp.Float('dropout_third', min_value=0.1, max_value=0.5, step=0.1)))

        # Output layer
        model.add(Dense(1, activation='sigmoid'))

        # Tuning the learning rate
        hp_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        
        model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        return model

Using TensorFlow backend


In [8]:
from kerastuner.tuners import Hyperband

hypermodel = MyHyperModel(input_dim=x_train.shape[1])

tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='keras_tuner_dir',
    project_name='keras_tuner'
)
from tensorflow.keras.callbacks import EarlyStopping

stop_early = EarlyStopping(monitor='val_loss', patience=5)
tuner.search(x_train, y_train,
             epochs=50,
             validation_split=0.1,
             callbacks=[stop_early])


Reloading Tuner from keras_tuner_dir\keras_tuner\tuner0.json


C:\Users\cmz\AppData\Local\Temp\ipykernel_19772\215952752.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import Hyperband


In [9]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)
# Summary of the best model
best_model.summary()
# Optionally, you can retrain the model with the best hyperparameters on the full dataset
best_model.fit(x_train, y_train, epochs=10, batch_size=8, validation_split=0.1)
y_pred_test = best_model.predict(x_test)
roc_auc_test = roc_auc_score(y_test, y_pred_test)
print(f'Best model AUC: {roc_auc_test}')


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 352)               2860000   
                                                                 
 dropout (Dropout)           (None, 352)               0         
                                                                 
 dense_4 (Dense)             (None, 160)               56480     
                                                                 
 dropout_1 (Dropout)         (None, 160)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                10304     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                

### RNN implementation

# Idea: 
- 2 LSTM branches-> iteratively takes in comment tokens, and parent comment tokens
- Dense branch-> dense layer takes in the other features
- Merge branches
- one more Dense layer


In [3]:
rnn_features = [
       'comment_word_count','parent_comment_word_count',
       'comment_token_count', 'parent_comment_token_count',
       'comment_unique_word_count', 'parent_comment_unique_word_count',
       'comment_unique_token_count', 'parent_comment_unique_token_count',
       'comment_stopword_count', 'parent_comment_stopword_count',
       'comment_mean_word_length', 'parent_comment_mean_word_length',
       'comment_mean_token_length', 'parent_comment_mean_token_length',
       'comment_char_count', 'parent_comment_char_count',
       'comment_punctuation_count', 'parent_comment_punctuation_count',
       'comment_hashtag_count', 'parent_comment_hashtag_count',
       'comment_number_count', 'parent_comment_number_count',
       'weighted_parent_sentiment_score_neutral',
       'weighted_parent_sentiment_score_positive',
       'weighted_comment_sentiment_score_neutral',
       'weighted_comment_sentiment_score_positive', 'documents_comment']#, 'documents_parent_comment'] #if doesnt work we try comment_tdidf_nn
train["documents_comment"] = train['comment_tokens'].apply(lambda tokens_list:' '.join(tokens_list)) #keras needs to use own tokenizer
#train["documents_parent_comment"] = train['parent_comment_tokens'].apply(lambda tokens_list:' '.join(tokens_list))
test["documents_comment"] = test['comment_tokens'].apply(lambda tokens_list:' '.join(tokens_list))
#test["documents_parent_comment"] = test['parent_comment_tokens'].apply(lambda tokens_list:' '.join(tokens_list))

rnn_train = train[rnn_features]
rnn_test = test[rnn_features]

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, TextVectorization
from tensorflow.keras.models import Model


# TextVectorization for comment and parent_comment
max_features = 8000 #follow number of tokens for feedforward


vectorize_layer_comment = TextVectorization(
    max_tokens=max_features,
    split='whitespace',
    ngrams=3
    )
# Prepare dataset for TextVectorization adapt
train_texts = rnn_train['documents_comment'].tolist()
test_texts = rnn_test['documents_comment'].tolist()
vectorize_layer_comment.adapt(train_texts)

In [6]:
from tensorflow.keras.layers import Embedding
#from tensorflow.keras.optimizers import Adam
#use tf.keras.optimizers.legacy.Adam if on M1/M2 macbook
# Assuming max_features is the vocabulary size and embedding_dim is the dimension of the embedding
embedding_dim = 128  # You can choose an appropriate value

# Add an Embedding layer after text vectorization
embedding_layer = Embedding(max_features, embedding_dim)

# LSTM Branch
text_input_comment = Input(shape=(1,), dtype=tf.string, name='text_comment')
text_features_comment = vectorize_layer_comment(text_input_comment)
text_features_comment = embedding_layer(text_features_comment)  # Embedding layer
lstm_comment = LSTM(64)(text_features_comment)

# Dense Features Branch
other_features_input = Input(shape=(len(rnn_features) - 1,), name='other_features')
dense_features = Dense(128, activation='relu')(other_features_input)

# Concatenate
concatenated = Concatenate()([lstm_comment, dense_features])

# Additional Dense Layers
output = Dense(64, activation='relu')(concatenated)
output = Dense(1, activation='sigmoid')(output)

# Build Model
model = Model(inputs=[text_input_comment, other_features_input], outputs=output)
#optimiser = tf.keras.optimizers.legacy.SGD(learning_rate=0.001)
# Compile
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])#use Adam optimiser


In [7]:

# Prepare text inputs
train_texts = np.array(train_texts)[:, np.newaxis]
test_texts = np.array(test_texts)[:, np.newaxis]

# Prepare other features inputs
feature_columns = [col for col in rnn_features if col != 'documents_comment']

train_other_features = np.array(rnn_train[feature_columns]).astype(np.float32)
test_other_features = np.array(rnn_test[feature_columns]).astype(np.float32)

# Fit the model
history = model.fit(
    [train_texts, train_other_features], y_train,
    epochs=40,
    batch_size=32,
    validation_split=0.25  # or use a validation set
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate([test_texts, test_other_features], y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict probabilities for the test set
y_pred_probs = model.predict([test_texts, test_other_features]).ravel()

# Calculate ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred_probs)
print(f"ROC-AUC Score: {roc_auc}")

Epoch 1/40
3774/3774 [==============================] - 104s 27ms/step - loss: 0.7698 - accuracy: 0.5429 - val_loss: 0.6793 - val_accuracy: 0.5651
Epoch 2/40
3774/3774 [==============================] - 98s 26ms/step - loss: 0.6876 - accuracy: 0.5586 - val_loss: 0.6800 - val_accuracy: 0.5580
Epoch 3/40
3774/3774 [==============================] - 101s 27ms/step - loss: 0.6844 - accuracy: 0.5662 - val_loss: 0.6767 - val_accuracy: 0.5656
Epoch 4/40
3774/3774 [==============================] - 102s 27ms/step - loss: 0.6766 - accuracy: 0.5671 - val_loss: 0.6774 - val_accuracy: 0.5625
Epoch 5/40
3774/3774 [==============================] - 99s 26ms/step - loss: 0.6753 - accuracy: 0.5698 - val_loss: 0.6753 - val_accuracy: 0.5688
Epoch 6/40
3774/3774 [==============================] - 95s 25ms/step - loss: 0.6748 - accuracy: 0.5710 - val_loss: 0.6739 - val_accuracy: 0.5711
Epoch 7/40
3774/3774 [==============================] - 96s 25ms/step - loss: 0.6829 - accuracy: 0.5703 - val_loss: 0.690